In [ ]:
import numpy as np
import glob
import shutil
import pandas as pd
import cv2
import os

In [ ]:
# 查询显卡信息
!nvidia-smi --query-gpu=name --format=csv,noheader

In [ ]:
# 更新pip
!pip install --upgrade pip

# 下载YOLOv5
!git config --global url."https://".insteadOf git://
!git clone git://github.com/ultralytics/yolov5.git

# 移动文件夹并安装yolo V5
!mv /kaggle/working/yolov5/* ./
!pip install -r requirements.txt
!pip uninstall -y wandb
!ls ./

In [ ]:
"""
# apex 安装cuda
!git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
"""

In [ ]:
# "A400M", "AG600", 
classes = ["A10","A400M", "AG600","B1","B2","B52", "Be200", "C130","C17","C5", "E2","EF2000",
           "F117","F14","F15","F16","F18","F22","F35","F4",
           "JAS39","MQ9","Mig31","Mirage2000","RQ4","Rafale",
           "SR71","Su57","Tu160","Tu95","U2","US2", "V22","XB70","YF23","J20"]
classes = np.array(classes)

In [ ]:
csv_paths = glob.glob('../input/militaryaircraftdetectiondataset/dataset/dataset/*.csv')
jpg_paths = glob.glob('../input/militaryaircraftdetectiondataset/dataset/dataset/*.jpg')
media_path = glob.glob('../input/test-media/Top 7 Badass Planes of the US Military-PU_OxwH1guM.mp4')
csv_paths.sort()
jpg_paths.sort()
print(csv_paths[0])
print(jpg_paths[0])
print(len(csv_paths), len(jpg_paths))

In [ ]:
os.makedirs('yolov5/data/train/images', exist_ok=True)
os.makedirs('yolov5/data/train/labels', exist_ok=True)

os.makedirs('yolov5/data/valid/images', exist_ok=True)
os.makedirs('yolov5/data/valid/labels', exist_ok=True)

os.makedirs('yolov5/data/test/images', exist_ok=True)
os.makedirs('yolov5/data/test/labels', exist_ok=True)

!ls yolov5/data

In [ ]:
n_images = len(csv_paths)
n_split = n_images//20

# shutil.copy('../input/yoloconfig/best0/best0.pt', 'yolov5/model/train/last.pt')
for i, (csv_path, jpg_path) in enumerate(zip(csv_paths, jpg_paths)):
    # 标定框数量
    annotations = np.array(pd.read_csv(csv_path))
    # 1份测试，4份验证，15份训练
    if i < n_split:
        shutil.copy(jpg_path, 'yolov5/data/test/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/test/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    elif n_split <= i < n_split*5:
        shutil.copy(jpg_path, 'yolov5/data/valid/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/valid/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    else:
        shutil.copy(jpg_path, 'yolov5/data/train/images/' + os.path.basename(jpg_path))
        txt_file_path = 'yolov5/data/train/labels/' + os.path.basename(csv_path)[:-4]+'.txt'
    
    
    
    with open(txt_file_path, mode='w') as f:
        for annotation in annotations:
            width = annotation[1]
            height = annotation[2]
            class_name = annotation[3]
            xmin = annotation[4]
            ymin = annotation[5]
            xmax = annotation[6]
            ymax = annotation[7]
            x_center = 0.5*(xmin+xmax)
            y_center = 0.5*(ymin+ymax)
            b_width = xmax - xmin
            b_height= ymax - ymin
            class_num = np.where(classes==class_name)
            try:
                class_num = class_num[0][0]
            except:
                print(class_name)
                continue
            output_string = '{} {} {} {} {}\n'.format(class_num,
                                                    x_center/width,
                                                    y_center/height,
                                                    b_width/width,
                                                    b_height/height)
            f.write(output_string)
print(len(glob.glob('yolov5/data/test/labels/*.txt')))
print(len(glob.glob('yolov5/data/valid/labels/*.txt')))
print(len(glob.glob('yolov5/data/train/labels/*.txt')))

In [ ]:
# train
%cd /kaggle/working
!python train.py --batch 88 \
                 --img 640 \
                 --epochs 50 \
                 --data ../input/yoloconfig/yolov5/data.yaml \
#                  --cfg ../input/yoloconfig/yolov5/yolov5x.yaml \
                 --hyp ../input/yoloconfig/yolov5/hyp.yaml \
                 --name military_aircraft_detection \
                 --weights /kaggle/input/yoloconfig/best_s/best_s.pt --device 0
#                  --weights ../input/yoloconfig/best2/best.pt --device 0

下面的代码用来将运行完的权重保存至云端数据集，用来下次调用继续训练

详细的API使用教程详见仓库：https://github.com/Kaggle/kaggle-api

In [ ]:
!cp /kaggle/working/runs/detect/exp2/001.mp4 /kaggle/working

In [ ]:
version = '001'
dataset_name = 'weights'
folder_name = 'MyWeight'
user_name = 'imchenyuxi'

In [ ]:
# create new folder
os.system(f'mkdir {folder_name}')
# move json file to before folder
os.system(f'cp -r ../input/{dataset_name}/kaggle.json /kaggle/working/{folder_name}') 
# move train result to folder
os.system(f'mv ./runs/train/* ./{folder_name}')
# Set base environment 
os.environ['KAGGLE_CONFIG_DIR'] = f'/kaggle/working/{folder_name}'
os.system(f'chmod 600 /kaggle/working/{folder_name}/kaggle.json')

# test Kaggle API condition
!kaggle competitions list

In [ ]:
import json
import datetime

def get_time_str():    
    curr_time = datetime.datetime.now()
    time = datetime.datetime.strftime(curr_time,'%m_%d_%H_%M_%S')
    return time

dataset = 'weights'
def sync():
    # Initiate
    os.system(f'kaggle datasets init -p {folder_name}')
    
    # Edit the dataset-metadata.json
    with open(f'./{folder_name}/dataset-metadata.json', "r",encoding='utf-8') as jsonFile:
        data = json.load(jsonFile)
    data["title"] = version
    data['id'] = f'{user_name}/{dataset}'
    with open(f'./{folder_name}/dataset-metadata.json', "w") as jsonFile:
        json.dump(data, jsonFile,ensure_ascii=False)
    # Upload your folder as a new version of the dataset
    # -m VERSION_NOTES     Message describing the new version
    # -p FOLDER    Folder for upload    
    # --dir-mode{skip,zip,tar}     means whether upload folder and compress file    
    # -d    Delete old versions of this dataset
    exit = os.system('kaggle datasets version --dir-mode tar -m' + get_time_str() + f' -p {folder_name}')
    
    if(exit == 0):
        print("Uploaded!")
    else:
        print("Failed to upload!, exit code ", exit)

sync()

In [ ]:
!rm -r ./output
!mv ./runs ./output